# Cargando lo que ya teníamos

In [1]:
import pandas as pd
import csv

In [2]:
df = pd.read_csv('corpus.csv')

## Distribución de palabras

Para entender un poco más el problema veremos la distribución de las palabras. Además esto nos va a ayudar a evitar problemas en un futuro.

In [3]:
from collections import Counter
from itertools import chain

In [14]:
distribution_words = Counter(chain.from_iterable(df.message.map(lambda x: x.split())))

In [15]:
df_distribution = pd.DataFrame(data={'word': list(distribution_words.keys()), 'counter': list(distribution_words.values())})

Es decir el vocabulario total es alrededor de:

In [16]:
df_distribution.count()

counter    101703
word       101703
dtype: int64

Por lo que las palabras menos vistas son

In [17]:
df_distribution.sort_values('counter').head(20)

,counter,word
50851,1,estaas
65599,1,boloooooooo
65598,1,Humble
65596,1,sin.hacer
65595,1,dinero.en
65594,1,muxo
65593,1,xno
65592,1,siendo.actor
65591,1,Jajaajjaaajajaja
65590,1,nickelodeon


Pero las más vistas son:

In [18]:
df_distribution.sort_values('counter', ascending=False).head(20)

,counter,word
11,24710,que
36,24339,de
15,19958,a
22,18941,la
45,17267,el
95,14857,no
30,13412,y
28,12577,en
2805,11407,omitido>
2804,11407,<Archivo


Al querer hacer un gráfico de caja, nos dimos cuenta que la media en la distribución de palabras era 1...

In [19]:
df_distribution.counter.median()

1.0

Pues luego de ver esa que la media era 1, nos preguntamos...
   - ¿Cuántas palabras hay con una sóla aparición? Son útiles o no?

In [20]:
sum(df_distribution.counter == 1) / df_distribution.count()

counter    0.648634
word       0.648634
dtype: float64

Vemos que hay alrededor de un 65% de palabras que aparecen una sóla vez... esto puede causar que la **vectorización no sea tan representativa como queremos...**

Problemas que vimos:
   - Al ser mensajes instantaneos, por lo general las personas no se preocupan con la ortográfia ni la forma de escribir.
   - Muchos tipos de mismas palabras, por ejemplo jajaja = jaja, dale = daleee.
   - _Archivo omitido_ como media no es útil.
   - Algunas puntaciones son inútiles para el problema (no ayudan en nada). Ejemplo quiero = quiero'
   - Los emoticones son necesarios para este problema?

## Conclusión

Después de discutir un rato decidimos tomar estás decisiones para mejorar la distribución:
   - "Normalizar palabras más vistas" (jajajajajaa --> ja)
   - Tratar las puntuaciones como palabras ("?", "!", "...") o directamente removerlas (quiero' --> quiero)
   - Pasar a minuscula toda palabra
   - Eliminar palabra que no aparezca más de 10 veces (soluciona errores de ortografía, links, etc)
   - Remover acentos
   - Pasar digito a la palabra NUM
   - Remover characteres repetidos, ejemplo: vaaaaamos por vamos.
   - __Hacer Stemm serviría? FIJARSE __